# Inversion of Frequency Domain Electromagnetic Data at Bookpurnong 

https://em.geosci.xyz/content/case_histories/bookpurnong/index.html

Lindsey J. Heagy, Rowan Cockett, Seogi Kang, Gudni K. Rosenkjaer, Douglas W. Oldenburg, A framework for simulation and inversion in electromagnetics, Computers & Geosciences, Volume 107, 2017, Pages 1-19, ISSN 0098-3004, http://dx.doi.org/10.1016/j.cageo.2017.06.018.

In [1]:
import dask
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.constants import mu_0
from scipy.spatial import cKDTree
import ipywidgets

import discretize
from pymatsolver import Pardiso
from SimPEG import (
    data, maps, utils,
    data_misfit, regularization,
    optimization,
    inversion, inverse_problem,
    directives,
)
from SimPEG.electromagnetics import frequency_domain as fdem

In [2]:
from matplotlib import rcParams
rcParams["font.size"] = 14

## Load and plot the data

In [3]:
data_directory = "bookpurnong-data"

In [4]:
# Load resolve data
resolve = h5py.File(os.path.sep.join([data_directory, "booky_resolve.hdf5"]), "r")

river_path = resolve["river_path"][()]  # River path
n_sounding = resolve["data"].shape[0]  # the # of soundings

# Bird height from surface
height_resolve = resolve["src_elevation"][()]

# fetch the frequencies we are considering
cpi_inds = [0, 2, 6, 8, 10]  # Indices for HCP in-phase
cpq_inds = [1, 3, 7, 9, 11]  # Indices for HCP quadrature
frequencies = resolve["frequency_cp"][()]

In [5]:
# plot observed and predicted data
def plot_data(frequency_index=0):

    fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    
    for i, a in enumerate(ax): 
        out = utils.plot2Ddata(
            resolve["xy"][:, :],
            resolve["data"][:, 2*frequency_index+i],
            ncontour=100,
            ax=a,
            
        )
        a.plot(resolve["xy"][:, 0], resolve["xy"][:, 1], 'k.', ms="2")
        cb = plt.colorbar(out[0], ax=a)
        cb.set_label("$\partial b/ \partial t$ (ppm)")

        header = str(resolve["data_header"][2*frequency_index + i])
        title = f"{header[5:-3]}Hz {'real' if header[4] == 'I' else 'imag'}"
        a.set_title(title)

        a.plot(river_path[:, 0], river_path[:, 1], "k-", lw=0.5)
        a.set_aspect(1)

        a.set_xlabel("easting (m)")
        a.set_ylabel("northing (m)")
    plt.tight_layout()

In [6]:
ipywidgets.interact(plot_data, frequency_index=ipywidgets.IntSlider(min=0, max=len(cpi_inds), value=0))

interactive(children=(IntSlider(value=0, description='frequency_index', max=5), Output()), _dom_classes=('widg…

<function __main__.plot_data(frequency_index=0)>

In [7]:
# survey parameters
rx_offset = 7.86  # tx-rx separation
bp = -mu_0 / (4 * np.pi * rx_offset ** 3)  # primary magnetic field

In [ ]:
@dask.delayed
def resolve_1Dinversions(
    dobs,
    src_height,
    frequencies,
    sigma_0,
    relative_error=0.08,
    floor=1e-14,
    rx_offset=7.86,
    beta=2.0,
    alpha_s=1e-3,
    alpha_x=1.0
    
):
    # ------------------- Mesh -------------------------------- #
    cs, ncx, ncz, npad = 1., 10., 10., 20
    pf = 1.3
    hx = [(cs, ncx), (cs, npad,pf)]
    npadz = 12
    hz = np.logspace(np.log10(1.0), np.log10(12.0), npad-1)
    hz_pad = hz[-1] * pf ** np.arange(npadz)
    hz = np.r_[hz_pad[::-1], hz[::-1], hz, hz_pad]
    mesh = discretize.CylMesh([hx, 1, hz], "00C")
    active = mesh.vectorCCz < 0.0
    
    # ------------------- Model & Mappings --------------------- #
    
    sig_air = 1e-8
    actMap = maps.InjectActiveCells(mesh, active, np.log(1e-8), nC=mesh.nCz)
    mapping = maps.ExpMap(mesh) * maps.SurjectVertical1D(mesh) * actMap
    m0 = np.log(sigma_0) * np.ones(active.sum())  # starting model

    # ------------------- Forward Simulation ------------------- #
    # set up the receivers
    receiver_list = [
        fdem.receivers.PointMagneticFluxDensitySecondary(
            np.array([[rx_offset, 0.0, src_height]]), orientation="z", component=component
        )
        for component in ["real", "imag"]
    ]

    # source location
    source_location = np.array([0.0, 0.0, src_height])
    source_list = [
        fdem.sources.MagDipole(receiver_list, frequency, source_location, orientation="z") 
        for frequency in frequencies
    ]

    # construct a forward simulation
    survey = fdem.Survey(source_list=source_list)
    simulation = fdem.Simulation3DMagneticFluxDensity(
        mesh, sigmaMap=mapping, solver=Pardiso
    )
    simulation.survey = survey

    # ------------------- Inversion ------------------- #
    # data misfit term
    uncertainty = abs(dobs) * relative + floor
    observed_data = data.Data(survey=survey, dobs=dobs, standard_deviation=uncertainty)
    dmisfit = data_misfit.L2DataMisfit(simulation=simulation, data=observed_data)

    # regularization
    regularization_mesh = discretize.TensorMesh([mesh.hz[mapping.maps[-1].indActive]])
    reg = regularization.Simple(regularization_mesh, alpha_s=alpha_s, alpha_x=alpha_x)

    # optimization
    opt = optimization.InexactGaussNewton(maxIter=10)

    # statement of the inverse problem
    inv_prob = inverse_problem.BaseInvProblem(dmisfit, reg, opt, beta=beta)

    # Inversion directives and parameters
    target = directives.TargetMisfit()
    inv = inversion.BaseInversion(inv_prob, directiveList=[target])

    # run the inversion
    mopt = inv.run(m0)
    return mopt, inv_prob.dpred, observed_data.dobs


In [ ]:
# build starting and reference model
sigma_half = 1e-1

# set up a noise model
# 10% for the 3 lowest frequencies, 15% for the two highest
relative = np.repeat(np.r_[np.ones(3) * 0.1, np.ones(2) * 0.15], 2)
floor = abs(20 * bp * 1e-6)  # floor of 20ppm

In [ ]:
# loop over the soundings and invert each
# initalize empty lists for storing inversion results
mopt = []  # recovered model
dpred = []  # predicted data
dobs = []  # observed data

for rxind in range(n_sounding):

    # convert data from ppm to magnetic field (A/m^2)
    dobsi = (
        np.c_[
            resolve["data"][rxind, :][cpi_inds].astype(float),
            resolve["data"][rxind, :][cpq_inds].astype(float),
        ].flatten()
        * bp
        * 1e-6
    )

    # perform the inversion
    src_height = height_resolve[rxind].astype(float)
    result = resolve_1Dinversions(
        dobsi,
        src_height,
        frequencies,
        sigma_0=sigma_half,
        relative_error=relative,
        floor=floor,
    )

    # add results to our list
    mopt.append(result[0])
    dpred.append(result[1])
    dobs.append(result[2])

In [ ]:
mopt = np.vstack(dask.compute(mopt))
# dpred = np.vstack(dpred)
# dobs = np.vstack(dobs)

In [ ]:
mopt

In [ ]:
sigma = np.exp(mopt_re)
indz = -7  # depth index

In [ ]:
# so that we can visually compare with literature (eg Viezzoli, 2010)
cmap = "jet"

# dummy figure for colobar
fig = plt.figure()
out = plt.scatter(np.ones(3), np.ones(3), c=np.linspace(-2, 1, 3), cmap=cmap)
plt.close(fig)

# plot from the paper
fs = 13  # fontsize
# matplotlib.rcParams['font.size'] = fs
plt.figure(figsize=(13, 7))
ax0 = plt.subplot2grid((2, 3), (0, 0), rowspan=2, colspan=2)
ax1 = plt.subplot2grid((2, 3), (0, 2))
ax2 = plt.subplot2grid((2, 3), (1, 2))

# titles of plots
title = [
    ("(a) Recovered model, %.1f m depth") % (-mesh.vectorCCz[active][indz]),
    "(b) Obs (Real 400 Hz)",
    "(c) Pred (Real 400 Hz)",
]

temp = sigma[:, indz]
tree = cKDTree(list(zip(resolve["xy"][:, 0], resolve["xy"][:, 1])))
d, d_inds = tree.query(list(zip(resolve["xy"][:, 0], resolve["xy"][:, 1])), k=20)
w = 1.0 / (d + 100.0) ** 2.0
w = utils.sdiag(1.0 / np.sum(w, axis=1)) * (w)
xy = resolve["xy"]
temp = (temp.flatten()[d_inds] * w).sum(axis=1)
utils.plot2Ddata(
    xy,
    temp,
    ncontour=100,
    scale="log",
    dataloc=False,
    contourOpts={"cmap": cmap, "vmin": 1e-2, "vmax": 1e1},
    ax=ax0,
)
ax0.plot(resolve["xy"][:, 0], resolve["xy"][:, 1], "k.", alpha=0.02, ms=1)

cb = plt.colorbar(out, ax=ax0, ticks=np.linspace(-2, 1, 4), format="$10^{%.1f}$")
cb.set_ticklabels(["0.01", "0.1", "1", "10"])
cb.set_label("Conductivity (S/m)")
ax0.plot(river_path[:, 0], river_path[:, 1], "k-", lw=0.5)

# plot observed and predicted data
freq_ind = 0
axs = [ax1, ax2]
temp_dobs = dobs_re[:, freq_ind].copy()
ax1.plot(river_path[:, 0], river_path[:, 1], "k-", lw=0.5)
inf = temp_dobs / abs(bp) * 1e6
print(inf.min(), inf.max())
out = utils.plot2Ddata(
    resolve["xy"][()],
    temp_dobs / abs(bp) * 1e6,
    ncontour=100,
    scale="log",
    dataloc=False,
    ax=ax1,
    contourOpts={"cmap": "viridis"},
)
vmin, vmax = out[0].get_clim()
print(vmin, vmax)
cb = plt.colorbar(
    out[0],
    ticks=np.logspace(np.log10(vmin), np.log10(vmax), 3),
    ax=ax1,
    format="%.1e",
    fraction=0.046,
    pad=0.04,
)
cb.set_label("Bz (ppm)")
temp_dpred = dpred_re[:, freq_ind].copy()
# temp_dpred[mask_:_data] = np.nan
ax2.plot(river_path[:, 0], river_path[:, 1], "k-", lw=0.5)
utils.plot2Ddata(
    resolve["xy"][()],
    temp_dpred / abs(bp) * 1e6,
    ncontour=100,
    scale="log",
    dataloc=False,
    contourOpts={"vmin": vmin, "vmax": vmax, "cmap": "viridis"},
    ax=ax2,
)
cb = plt.colorbar(
    out[0],
    ticks=np.logspace(np.log10(vmin), np.log10(vmax), 3),
    ax=ax2,
    format="%.1e",
    fraction=0.046,
    pad=0.04,
)
cb.set_label("Bz (ppm)")

for i, ax in enumerate([ax0, ax1, ax2]):
    xticks = [460000, 463000]
    yticks = [6195000, 6198000, 6201000]
    xloc, yloc = 462100.0, 6196500.0
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    # ax.plot(xloc, yloc, 'wo')
    ax.plot(river_path[:, 0], river_path[:, 1], "k", lw=0.5)

    ax.set_aspect("equal")
    ax.plot(resolve["xy"][:, 0], resolve["xy"][:, 1], "k.", alpha=0.02, ms=1)

    ax.set_yticklabels([str(f) for f in yticks])
    ax.set_ylabel("Northing (m)")
    ax.set_xlabel("Easting (m)")
    ax.set_title(title[i])

plt.tight_layout()